In [20]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [17]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df = pd.read_csv('/content/THAR-Dataset.csv')

In [5]:
df

,Identifier,Comment,SubTask1,SubTask2
0,3345,"चमचों! बोल दो वहां पाकिस्तान में भी ये सब ""भाज...",AntiReligion,Islam
1,9317,अब हम तुम जैसी औरतों पर भरोसा नहीं कर सकते हैं...,Non-AntiReligion,NaN
2,8713,"Ea log sach bolna, nhe jante, zhooth- per zhoo...",Non-AntiReligion,NaN
3,3072,Insaniyat naam ki chez nhi inme,Non-AntiReligion,NaN
4,5983,Bandar bajrang khan chale bare se muh ke sath ...,AntiReligion,Hinduism
...,...,...,...,...
11544,3215,इन लोगों को अपने कुरान का पुरा ज्ञान हि नहीं व...,AntiReligion,Islam
11545,10269,Aap sahi keh rahe hain...aur bhi achchha lagta...,AntiReligion,Hinduism
11546,2356,Jihadio ka sampurn bahiskar desh ke liye jaruri h,Non-AntiReligion,NaN
11547,6888,Bharat ke Hindu Yun Hi Soya Raha to Yahan Bhi ...,AntiReligion,Islam


In [6]:
df['SubTask1'] = df['SubTask1'].replace({'Non-AntiReligion':0, 'AntiReligion' : 1})

In [7]:
x = list(df['Comment'])
y_1 = list(df['SubTask1'])

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x,y_1, test_size = 0.2)

In [9]:
x_train_tokenized = tokenizer(x_train, padding = True, truncation = True, max_length = 512 )

In [10]:
x_val_tokenized = tokenizer(x_val, padding = True, truncation = True, max_length = 512 )

In [11]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [12]:
train_dataset = Dataset(x_train_tokenized, y_train)
val_dataset = Dataset(x_val_tokenized, y_val)

In [13]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [14]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [15]:
trainer.train()

Step,Training Loss
500,0.668400
1000,0.595500


TrainOutput(global_step=1155, training_loss=0.6276357873693689, metrics={'train_runtime': 866.7274, 'train_samples_per_second': 10.66, 'train_steps_per_second': 1.333, 'total_flos': 2430883040471040.0, 'train_loss': 0.6276357873693689, 'epoch': 1.0})

In [16]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.5846457481384277,
 'eval_accuracy': 0.6926406926406926,
 'eval_precision': 0.64171974522293,
 'eval_recall': 0.7560975609756098,
 'eval_f1': 0.6942291128337641,
 'eval_runtime': 67.206,
 'eval_samples_per_second': 34.372,
 'eval_steps_per_second': 4.3,
 'epoch': 1.0}

In [17]:
trainer.save_model('Bert-Subtask-1')